In [1]:
import numpy as np

In [2]:
def skew_matrix(v):
    matrix = [[0 for _ in range(3)]for _ in range(3)]
    matrix[1][0] = v[2]; matrix[0][1] = -v[2]
    matrix[2][0] = -v[1]; matrix[0][2] = v[1]
    matrix[2][1] = v[0]; matrix[1][2] = -v[0]
    return np.array(matrix)


def construct_matrix_A(pts1_norm , pts2_norm):
    n = pts1_norm.shape[0]
    A = np.zeros((n, 9))
    for i in range(n):
        x1, y1 = pts1_norm[i, :2]
        x2, y2 = pts2_norm[i, :2]
        A[i, :] = [x2*x1, x2*y1, x2, y2*x1, y2*y1, y2, x1, y1 , 1]
    return A


def eight_point_algo(pts1, pts2):
    A = construct_matrix_A(pts1 , pts2)
    _, _, V = np.linalg.svd(A)
    F = V[-1, :].reshape(3, 3)
    U, D, V = np.linalg.svd(F)
    D[-1] = 0
    F = np.dot(U, np.dot(np.diag(D), V))
    return F


In [3]:
def normalize_points(pts):
    n = pts.shape[0]
    mean = np.mean(pts, axis=0)
    s = np.sqrt(2) / np.linalg.norm(pts-mean,axis=1).mean()
    T = np.array([
    [s, 0, -s * mean[0]],
    [0, s, -s * mean[1]],
    [0, 0, 1]
    ])
    pts_norm = np.ones((n, 3))
    pts_norm[:, :2] = (pts - mean) * s
    return pts_norm , T

def denormalize_matrix(F_norm, pts1, pts2):
    T2, T1 = normalize_points(pts2)[1], normalize_points(pts1)[1]
    F = np.dot(T2.T, np.dot(F_norm, T1))
    return F / F[2, 2]


def eight_point_algo_robust(pts1, pts2):
    pts1_norm , _ = normalize_points(pts1) # 归一化
    pts2_norm , _ = normalize_points(pts2) # 归一化
    A = construct_matrix_A(pts1_norm , pts2_norm)
    _, _, V = np.linalg.svd(A)
    F = V[-1, :].reshape(3, 3)
    U, D, V = np.linalg.svd(F)
    D[-1] = 0
    F = np.dot(U, np.dot(np.diag(D), V))
    F = denormalize_matrix(F, pts1,pts2) # 反归一化
    return F


# 利用F矩阵计算E矩阵， 并计算RT
def essential_from_fundamental(F, K):
    E = np.dot(K.T, np.dot(F, K))
    U, D, V = np.linalg.svd(E)
    D = np.diag([1, 1, 0])
    E = np.dot(U, np.dot(D, V))
    return E/E[2,2]


def compute_rotation_translation(E):
    U, _, Vt = np.linalg.svd(E)
    if np.linalg.det(U) < 0:
        U[:, -1] *= -1
    if np.linalg.det(Vt) < 0:
        Vt[-1, :] *= -1
    W = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
    Wt = np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 1]])
    R1 = np.dot(U, np.dot(W, Vt))
    R2 = np.dot(U, np.dot(Wt, Vt))
    T1 = U[:, 2]
    T2 = -U[:,2]
    return R1, R2, T1,T2





# 三角化

def triangulate_dlt(K, R, T, points1 , points2):
    P1 = K@np.concatenate((np.eye(3),np.zeros((3,1))),axis=-1)
    P2 = K@np.concatenate((R,T.reshape(3,1)),axis=-1)
    num_points = points1.shape[0]
    points_3d = np.zeros((num_points , 3))
    for i in range(num_points):
        x1, y1 = points1[i]
        x2, y2 = points2[i]
        A = np.array([
                        [x1 * P1[2] - P1[0]],
                        [y1 * P1[2] - P1[1]],
                        [x2 * P2[2] - P2[0]],
                        [y2 * P2[2] - P2[1]]
                        ]).reshape(4, 4)
        _, _, V = np.linalg.svd(A)
        X_homogeneous = V[-1]
        X = X_homogeneous[:3] / X_homogeneous[3]
        points_3d[i] = X
    return points_3d

In [4]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from utils import*
from pysift import get_sift_points


data = np.load("assignment/pair1/data.npz")
x2d_0 = data['x2d_0']
x2d_1 = data['x2d_1']
K0 = np.loadtxt('assignment/pair1/K.txt', delimiter=',')

K = build_K(K0)

In [5]:
x2d_0

array([[ 594.84455279,  987.59363994],
       [   8.46389971, 1209.91461417],
       [ 419.25425162,   43.81932476],
       ...,
       [ 975.90274262,  450.82718935],
       [ 888.08209963,  529.33576063],
       [ 739.9499023 ,  554.28907395]])

In [6]:
x2d_0h = np.concatenate((x2d_0, np.ones((x2d_0.shape[0], 1))), axis=-1)
x2d_1h = np.concatenate((x2d_1, np.ones((x2d_1.shape[0], 1))), axis=-1)
x2d_0n = (np.linalg.inv(K)@(x2d_0h.T)).T
x2d_1n = (np.linalg.inv(K)@(x2d_1h.T)).T
E_best, mask = ransac_essential_matrix(x2d_0n, x2d_1n, threshold=0.5)
R1, R2, T1, T2 = compute_rotation_translation(E_best)

0.7130242825607064


In [7]:
x2d_0n

array([[-0.07895386,  0.12781838,  1.        ],
       [-0.28169027,  0.20492816,  1.        ],
       [-0.1396628 , -0.19952016,  1.        ],
       ...,
       [ 0.05279395, -0.05835361,  1.        ],
       [ 0.02243067, -0.03112371,  1.        ],
       [-0.02878485, -0.02246891,  1.        ]])

In [12]:
# 导入匹配点对
data = np.load('2d-matches.npz')
print(data)

In [13]:
# 利用匹配点计算 E 和 F 矩阵
x2d_0 = data['x2d_0']
x2d_1 = data['x2d_1']
K = data['K']
R0, T0 = data['R0'], data['T0']
R1, T1 = data['R1'], data['T1']

In [15]:
R = R1@R0.T
T = (T1-R@T0).flatten()
print(T)

[-0.973482  -1.1401584  0.7111603]


In [16]:

E = skew_matrix(T)@R
F_gt = np.linalg.inv(K.T)@E@np.linalg.inv(K)
F_gt /= F_gt[2, 2]


F_est =eight_point_algo(x2d_0, x2d_1)
F_est/= F_est[2,2]
print('八点法计算F矩阵:', F_est)
print('..............')
print('代数计算F矩阵:',F_gt)

八点法计算F矩阵: [[-3.73763673e-08 -1.50657477e-07 -5.91438393e-04]
 [-1.50412337e-07  3.81201053e-08  7.13092255e-04]
 [-7.19364663e-04 -3.19854570e-04  1.00000000e+00]]
..............
代数计算F矩阵: [[-3.73763879e-08 -1.50657447e-07 -5.91438529e-04]
 [-1.50412376e-07  3.81200158e-08  7.13092427e-04]
 [-7.19364572e-04 -3.19854662e-04  1.00000000e+00]]


In [17]:

points_3D = triangulate_dlt(K, R, T, x2d_0 , x2d_1)
print(points_3D)

[[-1.52927577e-01  3.15830360e-02  2.02966532e+00]
 [-1.50175645e-01 -2.30021917e-01  2.21910702e+00]
 [-4.45384592e-01 -7.86338233e-02  1.93245153e+00]
 [-4.30832469e-01 -5.88381862e-02  1.91162273e+00]
 [ 9.73059060e-03 -1.76223612e-01  2.60551877e+00]
 [-1.67341388e-01 -2.11589904e-01  2.01980380e+00]
 [-3.54210502e-03 -3.13681546e-01  2.25962441e+00]
 [-4.62886890e-01  3.89565778e-01  2.42112014e+00]
 [-3.06274388e-01 -1.82215440e-01  1.85511339e+00]
 [-5.74451164e-02  2.01082158e-01  2.32191709e+00]
 [-3.15388585e-01  1.14184037e-01  2.07629162e+00]
 [-2.63653041e-02 -7.23173606e-02  1.89641063e+00]
 [-1.68292313e-01 -1.73758868e-01  1.98891450e+00]
 [-4.04065763e-01  1.34010975e-03  1.91382867e+00]
 [-1.27004491e-01  1.38683654e-01  2.12621960e+00]
 [-9.05552977e-02  6.46598518e-02  2.12080626e+00]
 [-9.75658007e-02 -4.80748277e-02  1.87606241e+00]
 [-2.54112503e-01  2.68613439e-01  2.24339767e+00]
 [ 2.42292631e-01  1.81490210e-01  2.21528082e+00]
 [-9.80706322e-02  2.28066708e-

In [26]:

# 从投影点估计F矩阵

data = np.load('planar-case.npz')
K,R,T,X,im0,im1 = data['K'],data['R'],data['T'],data['X'],data['im0'],data['im1']
x2d_0 = K@X.T
x2d_0 = (x2d_0/x2d_0[-1]).T
x2d_1 = K@(R@X.T+T[:,None])
x2d_1 = (x2d_1/x2d_1[-1]).T
# F = eight_point_algo_robust(x2d_0[:8],x2d_1[:8])
F = eight_point_algo_robust(np.delete(x2d_0, -1, 1)[:8],np.delete(x2d_1, -1, 1)[:8])
print('从投影点估计F矩阵:', F)

从投影点估计F矩阵: [[ 1.96698443e-06 -1.74046705e-07 -3.16109380e-04]
 [ 1.25298055e-06  1.93137434e-06 -2.23087395e-03]
 [-2.29137199e-03 -4.32972754e-04  1.00000000e+00]]


In [19]:
# 四点法估计H矩阵
def four_point_homography(pts1, pts2):
    pts1_norm , T1 = normalize_points(pts1)
    pts2_norm , T2 = normalize_points(pts2)
    A = np.zeros((2*pts1_norm.shape[0], 9))
    for i in range(pts1_norm.shape[0]):
        x, y = pts1_norm[i][0], pts1_norm[i][1]
        u, v = pts2_norm[i][0], pts2_norm[i][1]
        A[2 * i] = [-x, -y, -1, 0, 0, 0, x * u, y * u, u]
        A[2 * i + 1] = [0, 0, 0, -x, -y, -1, x * v, y * v, v]
    # Solve for the homography matrix
    _, _, V = np.linalg.svd(A)
    H = V[-1].reshape((3, 3))
    # Denormalize the homography matrix
    H = np.dot(np.dot(np.linalg.inv(T2), H), T1)
    return H

data = np.load('planar-case.npz')
K,R,T,X,im0,im1 = data['K'],data['R'],data['T'],data['X'],data['im0'],data['im1']
x2d_0 = K@X.T
x2d_0 = (x2d_0/x2d_0[-1]).T
x2d_1 = K@(R@X.T+T[:,None])
x2d_1 = (x2d_1/x2d_1[-1]).T
H_uncalib = four_point_homography(np.delete(x2d_0, -1, 1),np.delete(x2d_1, -1, 1))
print(H_uncalib)

[[ 3.33515302e-01  2.84649560e-01 -9.72247459e+02]
 [-4.01669553e-01 -4.52639520e-03 -9.09276032e+01]
 [ 6.66566874e-05 -1.34614663e-04 -5.10184563e-01]]


In [20]:

def homography_calibrated(K, H_uncalib):
    H = np.linalg.inv(K) @ H_uncalib @ K
    eigvals = np.linalg.eigvals(H.T@H)
    H = H/np.sqrt(eigvals[1])
    return H

# H矩阵分解计算R，T
def get_motion_from_homography(H, K):
    H_calib = homography_calibrated(K, H)
    HtH = H_calib.T @ H_calib
    eigvals , eigvecs = np.linalg.eig(HtH)
    eigvals = eigvals[::-1]
    eigvecs = eigvecs[:, ::-1]
    v1 = eigvecs[:, 0]
    v2 = eigvecs[:, 1]
    u1 = (np.sqrt(1-eigvals[2])*eigvecs[:, 0]+np.sqrt(eigvals[0]-1)*eigvecs[:, 2])/np.sqrt(eigvals[0]-eigvals[2])
    u2 = (np.sqrt(1-eigvals[2])*eigvecs[:, 0]-np.sqrt(eigvals[0]-1)*eigvecs[:, 2])/np.sqrt(eigvals[0]-eigvals[2])
    U1 = np.stack([v2,u1,skew_matrix(v2)@u1],axis=1)
    W1 = np.stack([H_calib@v2 ,H_calib@u1 ,skew_matrix(H_calib@v2)@H_calib@u1],axis=1)
    U2 = np.stack([v2,u2,skew_matrix(v2)@u2],axis=1)
    W2 = np.stack([H_calib@v2 ,H_calib@u2 ,skew_matrix(H_calib@v2)@H_calib@u2],axis=1)
    R1 = W1@U1.T
    N1 = skew_matrix(v2)@u1
    T1 = (H_calib -R1)@N1
    R2 = W2@U2.T
    N2 = skew_matrix(v2)@u2
    T2 = (H_calib -R2)@N2
    R3 = R1
    N3 = -N1
    T3 = -T1
    R4 = R2
    N4 = -N2
    T4 = -T2
    return (R1,T1), (R2,T2), (R3,T3), (R4,T4)

In [21]:
RT_homo1 =get_motion_from_homography(H_uncalib ,K)
RT_homo2 =get_motion_from_homography(-H_uncalib ,K)
print(RT_homo1)

((array([[ 0.4986512 ,  0.70547193,  0.50364307],
       [-0.7951795 ,  0.14103535,  0.58974451],
       [ 0.34501672, -0.69456345,  0.63130426]]), array([0.55194075, 0.6461809 , 1.595253  ])), (array([[ 9.69991496e-01, -2.43136578e-01,  1.04960359e-03],
       [-2.43137872e-01, -9.69990786e-01,  1.35997714e-03],
       [ 6.87445628e-04, -1.57436464e-03, -9.99998524e-01]]), array([ 1.06040319,  1.24196194, -0.77465871])), (array([[ 0.4986512 ,  0.70547193,  0.50364307],
       [-0.7951795 ,  0.14103535,  0.58974451],
       [ 0.34501672, -0.69456345,  0.63130426]]), array([-0.55194075, -0.6461809 , -1.595253  ])), (array([[ 9.69991496e-01, -2.43136578e-01,  1.04960359e-03],
       [-2.43137872e-01, -9.69990786e-01,  1.35997714e-03],
       [ 6.87445628e-04, -1.57436464e-03, -9.99998524e-01]]), array([-1.06040319, -1.24196194,  0.77465871])))
